In [ ]:
#Package used in the exam
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from collections import Counter


import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.decomposition import PCA
import numpy.matlib
import matplotlib.patches as mpatches
from sklearn.cluster import KMeans

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import zero_one_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
#Loading the train and test data
train = np.loadtxt("VSTrain.dt", delimiter=",")
test = np.loadtxt("VSTest.dt", delimiter=",")

In [ ]:
# Splitting the training and test data into features and classes
x_train = train[:, :-1]
y_train = train[:, -1]

x_test = test[:, :-1]
y_test = test[:, -1]

#Counting the frequency divided with the total amount of data point for each class in training and test data
tmp = dict(Counter(y_train))
tmp1 = sorted(tmp)
for key in tmp1:
    print("Frequency for ", key, " divided with total amount of data points is ",tmp[key]/len(y_train))

print("---------------------------------------------")
tmp2 = dict(Counter(y_test))
tmp21 = sorted(tmp2)
for key in tmp21:
    print("Frequency for ", key, " divided with total amount of data points is ",tmp2[key]/len(y_train))


Frequency for  0.0  divided with total amount of data points is  0.08819714656290532
Frequency for  1.0  divided with total amount of data points is  0.028534370946822308
Frequency for  2.0  divided with total amount of data points is  0.0012970168612191958
Frequency for  3.0  divided with total amount of data points is  0.1245136186770428
Frequency for  4.0  divided with total amount of data points is  0.02204928664072633
Frequency for  5.0  divided with total amount of data points is  0.0648508430609598
Frequency for  6.0  divided with total amount of data points is  0.07782101167315175
Frequency for  7.0  divided with total amount of data points is  0.01297016861219196
Frequency for  8.0  divided with total amount of data points is  0.03501945525291829
Frequency for  9.0  divided with total amount of data points is  0.07522697795071336
Frequency for  10.0  divided with total amount of data points is  0.011673151750972763
Frequency for  11.0  divided with total amount of data points 

In [ ]:
# printing the amount of examples each class will have after removel of data points
train_arr=[]
for key in tmp:
    if(tmp[key]>64):
        train_arr.append(key)
        print("Label ",key," has", tmp[key], " training examples and ", tmp2[key], " test examples")

print(train_arr)

# choosing the classes with 65 or more data points
x_train1 = []
y_train1 = []

x_test1 = []
y_test1 = []

for i in range(len(y_train)):
    for k in range(len(train_arr)):
        if (y_train[i] == train_arr[k]):
            x_train1.append(x_train[i])
            y_train1.append(y_train[i])

for i in range(len(y_test)):
    for k in range(len(train_arr)):
        if (y_test[i] == train_arr[k]):
            x_test1.append(x_test[i])
            y_test1.append(y_test[i])

x1 = np.array(x_train1) 
y1 = np.array(y_train1) 

xtest1 = np.array(x_test1)
ytest1 = np.array(y_test1)



Label  0.0  has 68  training examples and  76  test examples
Label  22.0  has 82  training examples and  87  test examples
Label  23.0  has 68  training examples and  77  test examples
Label  3.0  has 96  training examples and  95  test examples
[0.0, 22.0, 23.0, 3.0]


In [ ]:
#Normalizing the data. Same functions as in assignment 4
mean = []
var = []
for i in range(x1.shape[1]):
    tmp = x1[:,i]
    mean.append(np.mean(tmp))
    var.append(np.var(tmp))

normal = np.zeros((314, 61), dtype=object)

def normal1(data):
    x_train1 = np.zeros_like(data)

    for i in range(data.shape[1]):
        tmp1 = data[:,i]
        x_train1[:,i] = (tmp1-mean[i])/np.sqrt(var[i])
        normal[i,0] = "%.5f" % np.mean(x_train1[:, i])
        normal[i,1] = "%.5f" % np.var(x_train1[:, i])
        #print("Mean: ", normal[i,0]," Var: ", normal[i,1])
    return x_train1

normal1(x1)

array([[ 3.01752054, -0.11003752,  0.39324135, ...,  2.49739366,
         2.61870994,  0.39958652],
       [-0.66660712, -0.11003752, -0.68802212, ..., -0.64611458,
        -0.68630225, -0.35443278],
       [ 0.43724228, -0.11003752,  3.48402642, ...,  0.63904954,
         0.77693232,  0.15071849],
       ...,
       [-0.52691002, -0.11003752,  0.44494398, ..., -0.61362851,
        -0.63349213,  0.0338409 ],
       [ 0.178843  , 10.46977656, -0.80195904, ..., -0.15573477,
        -0.44131782, -2.0395886 ],
       [-0.30468664, -0.11003752,  0.80820035, ..., -0.34664124,
        -0.20796884,  1.16773001]])

In [ ]:
#Making the PCA analysis
pca = PCA()
pC = pca.fit_transform(normal1(x1))
x_coor, y_coor = pC[:,0], pC[:,1]

#Plotting the result as scatter plot
plt.scatter(x_coor, y_coor, c=['b' if i==0 else 'r' if i ==22 else 'black' if i==23 else 'g' for i in y1], s=10)
d1 = mpatches.Patch(color='b', label='label 0')
d2 = mpatches.Patch(color='r', label='label 22')
d3 = mpatches.Patch(color='black', label='label 23')
d4 = mpatches.Patch(color='g', label='label 3')
plt.legend(handles=[d1, d2, d3, d4])
plt.title("First two principal components")
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
plt.savefig("2.png")
plt.cla()
plt.clf()

<Figure size 432x288 with 0 Axes>

In [ ]:
# using kMeans on the data
kmeans = KMeans(random_state=0,n_clusters=4).fit(pC)
k_centers = kmeans.cluster_centers_

pC1 = pca.fit_transform(k_centers)
x1_coor, y1_coor = pC1[:,0], pC1[:,1]
#Plotting the result on top of the scatter plot with the PCA analysis
plt.scatter(x_coor, y_coor, c=['b' if i==0 else 'r' if i ==22 else 'black' if i==23 else 'g' for i in y1], s=10)
plt.scatter(x1_coor, y1_coor, c='purple',marker='D', s=80)
d5 = mpatches.Patch(color='purple', label='cluster centers')
plt.legend(handles=[d1, d2, d3, d4, d5])
plt.title("First two principal components")
plt.xlabel("First principal component")
plt.ylabel("Second principal component")
plt.savefig("3.png")
plt.cla()
plt.clf()

<Figure size 432x288 with 0 Axes>

In [ ]:
#Using the logistic regression model
logReg = LogisticRegression(multi_class='multinomial',solver ='newton-cg').fit(x1, y1)

yhat = logReg.predict(x1)
yhat1 = logReg.predict(x_test1)

from sklearn.metrics import accuracy_score
# Printing the zero one loss result
print(zero_one_loss(y1, yhat))
print(zero_one_loss(y_test1, yhat1))

0.031847133757961776
0.11641791044776117


In [ ]:
# Using the random forest classifier with square root of features
clf = RandomForestClassifier(n_estimators=200, random_state=0,max_features='auto', oob_score=True)
clf.fit(x1, y1)
grid_predictions1 = clf.predict(x1)
# Printing the zero one loss result
print(zero_one_loss(y1, grid_predictions1)) 
print(1 - clf.oob_score_)

clf1 = RandomForestClassifier(n_estimators=200, random_state=0,max_features='auto', oob_score=True)
clf1.fit(x1, y1)
grid_predictions11 = clf1.predict(x_test1)
# Printing the zero one loss result
print(zero_one_loss(y_test1, grid_predictions11))
print(1 - clf1.oob_score_)

0.0
0.06369426751592355
0.08358208955223878
0.06369426751592355


In [ ]:
# Using the random forest classifier with all features 
clf2 = RandomForestClassifier(n_estimators=200, random_state=0,max_features=61, oob_score=True)
clf2.fit(x1, y1)
grid_predictions2 = clf2.predict(x1)
# Printing the zero one loss result
print(zero_one_loss(y1, grid_predictions2))
print(1 - clf2.oob_score_)


clf3 = RandomForestClassifier(n_estimators=200, random_state=0,max_features=61, oob_score=True)
clf3.fit(x1, y1)
grid_predictions22 = clf3.predict(x_test1)
# Printing the zero one loss result
print(zero_one_loss(y_test1, grid_predictions22))
print(1 - clf3.oob_score_)

0.0
0.08280254777070062
0.06865671641791049
0.08280254777070062


In [ ]:
# using the k-nearest neighbor classifier with cross-validation 
knn = KNeighborsClassifier()
p = {'n_neighbors': np.arange(1, 20)}
grid = GridSearchCV(knn, p, cv = 10)
grid.fit(x1, y1)
bestK = grid.best_params_
bestK1 = bestK.values()
K1List = list(bestK1)
kNeigh = K1List[0]
print('best k = ', kNeigh)


pred = grid.predict(x1)
#print("Result of the KNN classifier: ", score)
print("0-1 loss train = ", zero_one_loss(y1, pred))


pred1 = grid.predict(x_test1)
#print("Result of the KNN classifier: ", score)
print("0-1 loss test = ", zero_one_loss(y_test1, pred1))

best k =  8
0-1 loss train =  0.2993630573248408
0-1 loss test =  0.34925373134328364
